# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [14]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17425140732486632524
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 313393152
locality {
  bus_id: 1
}
incarnation: 9049796472155798783
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [17]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [18]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [19]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [20]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    tokenizer_inputs = Tokenizer()
    tokenizer_inputs.fit_on_texts(x)
    x = tokenizer_inputs.texts_to_sequences(x)
    
    return x, tokenizer_inputs
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [21]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max(len(s) for s in x)
    x = pad_sequences(x, maxlen=length, padding='post')
    return x

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [22]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [23]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_sequence = Input(shape=input_shape[1:] )
#     embedding_layer = Embedding(
#                       english_vocab_size+1,
#                       english_vocab_size+1,
#                       weights = [np.diag(np.ones(english_vocab_size+1))],
#                       trainable=False
#                         )
#     x = embedding_layer(input_sequence)
#     gru = GRU(units=english_vocab_size, return_sequences=True)
#     rnn_outputs = gru(x)
#     dense = Dense(french_vocab_size, activation='softmax')
#     logits = dense(rnn_outputs)
                             
#     model = Model(input_sequence, logits)

#     model.compile(loss='categorical_crossentropy',
#                   optimizer=Adam(lr=1e-3),
#                   metrics=['accuracy'])
    
    rnn_outputs = GRU(units=english_vocab_size, return_sequences=True)(input_sequence)
    logits = TimeDistributed(Dense(units=french_vocab_size))(rnn_outputs) 
                             
    model = Model(input_sequence, Activation('softmax')(logits))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=1e-3),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 12s 107us/step - loss: 2.6893 - acc: 0.4714 - val_loss: nan - val_acc: 0.5531
Epoch 2/10
110288/110288 [==============================] - 10s 87us/step - loss: 1.7232 - acc: 0.5780 - val_loss: nan - val_acc: 0.5932
Epoch 3/10
110288/110288 [==============================] - 10s 88us/step - loss: 1.4878 - acc: 0.6081 - val_loss: nan - val_acc: 0.6170
Epoch 4/10
110288/110288 [==============================] - 10s 88us/step - loss: 1.3682 - acc: 0.6248 - val_loss: nan - val_acc: 0.6329
Epoch 5/10
110288/110288 [==============================] - 10s 88us/step - loss: 1.2740 - acc: 0.6408 - val_loss: nan - val_acc: 0.6457
Epoch 6/10
110288/110288 [==============================] - 10s 88us/step - loss: 1.2022 - acc: 0.6516 - val_loss: nan - val_acc: 0.6561
Epoch 7/10
110288/110288 [==============================] - 10s 88us/step - loss: 1.1488 - acc: 0.6595 - val_loss: nan - val_a

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [27]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    dropout = 0.5
    inputshape = input_shape[1:]
    embeddim = 200
    outputdim = french_vocab_size
    rnnunits = 128
    input_sequence = Input(shape=inputshape)
    embedding_layer = Embedding(
                      english_vocab_size,
                      embeddim,
                      embeddings_initializer="glorot_normal",
                      input_length=input_shape[1:][0],
                      trainable=True
                        )
    x = embedding_layer(input_sequence)
    
    gru = GRU(units=rnnunits, dropout=dropout, return_sequences=True) 
    rnn_outputs = gru(x)
    dense = TimeDistributed(Dense(french_vocab_size , activation='softmax'))
    logits = dense(rnn_outputs)
                             
    model = Model(input_sequence, logits)

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=1e-3),
                  metrics=['accuracy'])
        
    
    return model  
tests.test_embed_model(embed_model)


# Pad the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=100, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/100
110288/110288 [==============================] - 10s 90us/step - loss: 3.2120 - acc: 0.4428 - val_loss: 2.3341 - val_acc: 0.5309
Epoch 2/100
110288/110288 [==============================] - 9s 84us/step - loss: 1.7870 - acc: 0.5969 - val_loss: 1.3055 - val_acc: 0.6800
Epoch 3/100
110288/110288 [==============================] - 9s 84us/step - loss: 1.0999 - acc: 0.7256 - val_loss: 0.8749 - val_acc: 0.7734
Epoch 4/100
110288/110288 [==============================] - 9s 84us/step - loss: 0.8211 - acc: 0.7784 - val_loss: 0.6838 - val_acc: 0.8078
Epoch 5/100
110288/110288 [==============================] - 9s 84us/step - loss: 0.6740 - acc: 0.8072 - val_loss: 0.5765 - val_acc: 0.8301
Epoch 6/100
110288/110288 [==============================] - 9s 84us/step - loss: 0.5793 - acc: 0.8281 - val_loss: 0.4944 - val_acc: 0.8530
Epoch 7/100
110288/110288 [==============================] - 9s 84us/step - loss: 0.5131 - acc: 0.8449 - val

Epoch 59/100
110288/110288 [==============================] - 9s 84us/step - loss: 0.1930 - acc: 0.9327 - val_loss: 0.1859 - val_acc: 0.9359
Epoch 60/100
110288/110288 [==============================] - 9s 84us/step - loss: 0.1922 - acc: 0.9328 - val_loss: 0.1857 - val_acc: 0.9360
Epoch 61/100
110288/110288 [==============================] - 9s 84us/step - loss: 0.1908 - acc: 0.9331 - val_loss: 0.1848 - val_acc: 0.9361
Epoch 62/100
110288/110288 [==============================] - 9s 84us/step - loss: 0.1909 - acc: 0.9330 - val_loss: 0.1856 - val_acc: 0.9358
Epoch 63/100
110288/110288 [==============================] - 9s 84us/step - loss: 0.1905 - acc: 0.9331 - val_loss: 0.1844 - val_acc: 0.9365
Epoch 64/100
110288/110288 [==============================] - 9s 84us/step - loss: 0.1894 - acc: 0.9334 - val_loss: 0.1838 - val_acc: 0.9364
Epoch 65/100
110288/110288 [==============================] - 9s 84us/step - loss: 0.1893 - acc: 0.9335 - val_loss: 0.1832 - val_acc: 0.9366
Epoch 66/100


In [28]:
english_sentences[1]

'the united states is usually chilly during july , and it is usually freezing in november .'

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [29]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    rnnunits = 128
    inputshape = input_shape[1:]
    outputdim = french_vocab_size
    dropout = 0.5
    input_sequence = Input(shape=inputshape)
    
    rnn_outputs = Bidirectional(GRU(units=rnnunits, dropout= dropout, return_sequences=True))(input_sequence)
    logits = TimeDistributed(Dense(units=outputdim))(rnn_outputs) 
                             
    model = Model(input_sequence, Activation('softmax')(logits))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=1e-3),
                  metrics=['accuracy'])
    
    return model

tests.test_bd_model(bd_model)


tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network 
bd_rnn_model = bd_model(input_shape = tmp_x.shape,
                           output_sequence_length = max_french_sequence_length,
                           english_vocab_size = english_vocab_size+1,
                           french_vocab_size = french_vocab_size+1)


bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=100, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/100
110288/110288 [==============================] - 13s 118us/step - loss: 3.0344 - acc: 0.4672 - val_loss: 2.1028 - val_acc: 0.5203
Epoch 2/100
110288/110288 [==============================] - 12s 109us/step - loss: 1.9702 - acc: 0.5422 - val_loss: 1.7492 - val_acc: 0.5628
Epoch 3/100
110288/110288 [==============================] - 12s 109us/step - loss: 1.7290 - acc: 0.5708 - val_loss: 1.6633 - val_acc: 0.5685
Epoch 4/100
110288/110288 [==============================] - 12s 110us/step - loss: 1.6273 - acc: 0.5865 - val_loss: 1.6514 - val_acc: 0.5632
Epoch 5/100
110288/110288 [==============================] - 12s 109us/step - loss: 1.5554 - acc: 0.5977 - val_loss: 1.6785 - val_acc: 0.5609
Epoch 6/100
110288/110288 [==============================] - 12s 110us/step - loss: 1.5000 - acc: 0.6070 - val_loss: 1.7198 - val_acc: 0.5575
Epoch 7/100
110288/110288 [==============================] - 12s 109us/step - loss: 1.4559 - acc:

Epoch 58/100
110288/110288 [==============================] - 12s 109us/step - loss: 1.0086 - acc: 0.6899 - val_loss: 3.7669 - val_acc: 0.5020
Epoch 59/100
110288/110288 [==============================] - 12s 110us/step - loss: 1.0065 - acc: 0.6901 - val_loss: 3.7809 - val_acc: 0.5011
Epoch 60/100
110288/110288 [==============================] - 12s 110us/step - loss: 1.0051 - acc: 0.6904 - val_loss: 3.8122 - val_acc: 0.5008
Epoch 61/100
110288/110288 [==============================] - 12s 110us/step - loss: 1.0023 - acc: 0.6910 - val_loss: 3.8194 - val_acc: 0.5013
Epoch 62/100
110288/110288 [==============================] - 12s 110us/step - loss: 0.9999 - acc: 0.6913 - val_loss: 3.8177 - val_acc: 0.5014
Epoch 63/100
110288/110288 [==============================] - 12s 110us/step - loss: 0.9986 - acc: 0.6916 - val_loss: 3.8622 - val_acc: 0.5012
Epoch 64/100
110288/110288 [==============================] - 12s 110us/step - loss: 0.9962 - acc: 0.6926 - val_loss: 3.8663 - val_acc: 0.5011

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [37]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    e_rnn_cells = 200
    d_rnn_cells = 264
    input_sequence = Input(shape=input_shape[1:])
    encoder = GRU(units=e_rnn_cells, return_state=True, return_sequences=False)
    encoder_output, encoder_state = encoder(input_sequence)
    encoder_state_projection = Dense(units = d_rnn_cells)
    encoder_state = encoder_state_projection(encoder_state)
    repeat = RepeatVector(output_sequence_length)
    matrix_rep = repeat(encoder_output)
    decoder = GRU(units=d_rnn_cells, return_state=False, return_sequences=True)
    decoder_output = decoder(matrix_rep, initial_state=encoder_state)
    
    dense = Dense(units=french_vocab_size, activation='softmax')
    logits = dense(decoder_output)
    model = Model(input_sequence, logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=1e-3),
                  metrics=['accuracy'])
    
    return model

# Unitary tests
tests.test_encdec_model(encdec_model)


# Train and Print prediction(s)

# Pad and Reshape the input to work with the Embedding layer
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network 
encdec_rnn_model = encdec_model(input_shape = tmp_x.shape,
                                output_sequence_length = max_french_sequence_length,
                                english_vocab_size = english_vocab_size+1,
                                french_vocab_size = french_vocab_size+1)

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 20s 183us/step - loss: 2.6884 - acc: 0.4801 - val_loss: 1.9727 - val_acc: 0.5403
Epoch 2/10
110288/110288 [==============================] - 19s 173us/step - loss: 1.8073 - acc: 0.5513 - val_loss: 1.5783 - val_acc: 0.5800
Epoch 3/10
110288/110288 [==============================] - 19s 173us/step - loss: 1.5079 - acc: 0.5924 - val_loss: 1.4404 - val_acc: 0.6038
Epoch 4/10
110288/110288 [==============================] - 19s 173us/step - loss: 1.4115 - acc: 0.6089 - val_loss: 1.3687 - val_acc: 0.6185
Epoch 5/10
110288/110288 [==============================] - 19s 173us/step - loss: 1.3450 - acc: 0.6195 - val_loss: 1.3060 - val_acc: 0.6278
Epoch 6/10
110288/110288 [==============================] - 19s 173us/step - loss: 1.3094 - acc: 0.6246 - val_loss: 1.2800 - val_acc: 0.6286
Epoch 7/10
110288/110288 [==============================] - 19s 173us/step - loss: 1.2594 - acc: 0.6341

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [68]:
from keras.layers import LSTM, concatenate, Lambda # Going to use Bidirectional LSTM
import keras.backend as K

In [78]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    def multiply(x):
        outputseq,mask = x
        mask = K.stack([mask]*d_rnn_cells, axis=-1) 
        return mask*outputseq
    
    e_rnn_cells = 128
    d_rnn_cells = 2 * e_rnn_cells
    learning_rate = 1e-3
    inputshape = input_shape[1:]
    embeddim = 200
    outputdim = french_vocab_size
    input_sequence = Input(shape=inputshape)
    embedding_layer = Embedding(
                      english_vocab_size,
                      embeddim,
                      embeddings_initializer="glorot_normal",
                      input_length=input_shape[1:][0],
                      trainable=True
                        )
    x = embedding_layer(input_sequence)
    # Encoder backward RNN layer
    encoder_forward_output, forward_state_h, forward_state_c = LSTM(units=e_rnn_cells,
                                                                    
                                                                    return_sequences=False,
                                                                    return_state=True,
                                                                    go_backwards=False)(x)
    
    # Encoder backward RNN layer
    encoder_backward_output, backward_state_h, backward_state_c = LSTM(units=e_rnn_cells,
                                                                       
                                                                       return_sequences=False,
                                                                       return_state=True,
                                                                       go_backwards=True)(x)
    
    # Encoder output and states : Merge the LSTM Forward and Backward ouputs (using 'concatenate' method) 
    state_h = concatenate([forward_state_h, backward_state_h]) 
    state_c = concatenate([forward_state_c, backward_state_c])    
    encoder_output = concatenate([encoder_forward_output, encoder_backward_output])   
    
    # Not Attention but just some one time weights on encoder_outputs
    notAttention = Dense(output_sequence_length, activation='softmax')
    weights = notAttention(encoder_output)
    repeat = RepeatVector(output_sequence_length)
    matrix_rep = repeat(encoder_output)

    out = Lambda(multiply)([matrix_rep,weights])

    
    decoder_output = LSTM(units=d_rnn_cells,
                          return_sequences=True,
                          return_state=False,
                          go_backwards=False)(out, initial_state=[state_h, state_c])
    
    # Decoder output
    logits = TimeDistributed(Dense(units=french_vocab_size))(decoder_output) 
    
    # Model
    model = Model(input_sequence, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    
    return model
    
tests.test_model_final(model_final)


print('Final Model Loaded')

# Train and Print prediction(s)

# Pad the input to work with the Embedding layer
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)


# Train the neural network 
final_rnn_model = model_final(input_shape = tmp_x.shape,
                              output_sequence_length = max_french_sequence_length,
                              english_vocab_size = english_vocab_size+1,
                              french_vocab_size = french_vocab_size+1)

print(final_rnn_model.summary(line_length=125))


final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=100, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
_____________________________________________________________________________________________________________________________
Layer (type)                             Output Shape               Param #        Connected to                              
input_49 (InputLayer)                    (None, 21)                 0                                                        
_____________________________________________________________________________________________________________________________
embedding_34 (Embedding)                 (None, 21, 200)            40000          input_49[0][0]                            
_____________________________________________________________________________________________________________________________
lstm_50 (LSTM)                           [(None, 128), (None, 128), 168448         embedding_34[0][0]                        
___________________________________________________________________________________________________

110288/110288 [==============================] - 31s 279us/step - loss: 0.1183 - acc: 0.9666 - val_loss: 0.1274 - val_acc: 0.9642
Epoch 26/100
110288/110288 [==============================] - 31s 278us/step - loss: 0.1154 - acc: 0.9673 - val_loss: 0.1202 - val_acc: 0.9660
Epoch 27/100
110288/110288 [==============================] - 31s 279us/step - loss: 0.1058 - acc: 0.9698 - val_loss: 0.1159 - val_acc: 0.9667
Epoch 28/100
110288/110288 [==============================] - 31s 278us/step - loss: 0.0992 - acc: 0.9718 - val_loss: 0.1142 - val_acc: 0.9680
Epoch 29/100
110288/110288 [==============================] - 31s 279us/step - loss: 0.0985 - acc: 0.9717 - val_loss: 0.1120 - val_acc: 0.9681
Epoch 30/100
110288/110288 [==============================] - 31s 279us/step - loss: 0.0955 - acc: 0.9722 - val_loss: 0.1066 - val_acc: 0.9694
Epoch 31/100
110288/110288 [==============================] - 31s 279us/step - loss: 0.0982 - acc: 0.9717 - val_loss: 0.1056 - val_acc: 0.9690
Epoch 32/100

110288/110288 [==============================] - 31s 280us/step - loss: 0.0343 - acc: 0.9902 - val_loss: 0.0641 - val_acc: 0.9834
Epoch 83/100
110288/110288 [==============================] - 31s 279us/step - loss: 0.0204 - acc: 0.9941 - val_loss: 0.0597 - val_acc: 0.9848
Epoch 84/100
110288/110288 [==============================] - 31s 279us/step - loss: 0.0218 - acc: 0.9937 - val_loss: 0.0630 - val_acc: 0.9837
Epoch 85/100
110288/110288 [==============================] - 31s 279us/step - loss: 0.0188 - acc: 0.9946 - val_loss: 0.0583 - val_acc: 0.9852
Epoch 86/100
110288/110288 [==============================] - 31s 279us/step - loss: 0.0175 - acc: 0.9951 - val_loss: 0.0624 - val_acc: 0.9846
Epoch 87/100
110288/110288 [==============================] - 31s 278us/step - loss: 0.0183 - acc: 0.9947 - val_loss: 0.0653 - val_acc: 0.9836
Epoch 88/100
110288/110288 [==============================] - 31s 279us/step - loss: 0.0169 - acc: 0.9952 - val_loss: 0.0572 - val_acc: 0.9861
Epoch 89/100

## Prediction (IMPLEMENTATION)

In [79]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(input_shape = x.shape,
                        output_sequence_length = y.shape[1],
                        english_vocab_size = len(x_tk.word_index)+1,
                        french_vocab_size = len(y_tk.word_index)+1)

    model.fit(x, y, batch_size=1024, epochs=30, validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 29s 267us/step - loss: 2.7502 - acc: 0.4810 - val_loss: 1.9828 - val_acc: 0.5549
Epoch 2/30
110288/110288 [==============================] - 27s 243us/step - loss: 1.6505 - acc: 0.5956 - val_loss: 1.4120 - val_acc: 0.6364
Epoch 3/30
110288/110288 [==============================] - 27s 243us/step - loss: 1.2797 - acc: 0.6706 - val_loss: 1.1525 - val_acc: 0.6990
Epoch 4/30
110288/110288 [==============================] - 27s 244us/step - loss: 1.0602 - acc: 0.7193 - val_loss: 0.9734 - val_acc: 0.7394
Epoch 5/30
110288/110288 [==============================] - 27s 245us/step - loss: 0.9051 - acc: 0.7539 - val_loss: 0.8410 - val_acc: 0.7688
Epoch 6/30
110288/110288 [==============================] - 27s 244us/step - loss: 0.7876 - acc: 0.7806 - val_loss: 0.7389 - val_acc: 0.7921
Epoch 7/30
110288/110288 [==============================] - 27s 244us/step - loss: 0.6941 - acc: 0.8034

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [80]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 430600 bytes to machine_translation.html',
 '[NbConvertApp] Converting notebook machine_translation-zh.ipynb to html',
 '[NbConvertApp] Writing 328614 bytes to machine_translation-zh.html',
 '[NbConvertApp] Converting notebook Seq2Seq-EncoderDecoder-TheProperWay-Keras.ipynb to html',
 '[NbConvertApp] Writing 320899 bytes to Seq2Seq-EncoderDecoder-TheProperWay-Keras.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?